# Loop 6 Analysis: Understanding the Gap and Finding a Path Forward

## Key Observations:
1. 6 experiments, ALL at the same score (70.734327)
2. Target is 68.931058 - need 1.8 points improvement (~2.5%)
3. The evaluator correctly identified that parameters used were too low
4. The jiweiliu kernel shows a promising approach: SA with translations + deletion cascade

In [ ]:
import pandas as pd
import numpy as np
import os
import glob

# Check what CSVs are available
print('=== Available CSV files ===')
csv_files = glob.glob('/home/nonroot/snapshots/santa-2025/*/code/datasets/**/*.csv', recursive=True)
for f in csv_files[:20]:
    print(f)

In [ ]:
# Load baseline and check per-N scores
from numba import njit
import math

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def get_bbox(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    side = get_bbox(xs, ys, degs, tx, ty)
    return side * side / n

tx, ty = make_polygon_template()

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

# Load baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv'
df = pd.read_csv(baseline_path)
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

per_n_scores = {}
for n, g in df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    per_n_scores[n] = score_group(xs, ys, ds, tx, ty)

print(f'Total baseline score: {sum(per_n_scores.values()):.6f}')
print(f'Target: 68.931058')
print(f'Gap: {sum(per_n_scores.values()) - 68.931058:.6f}')

In [ ]:
# Analyze which N values contribute most to the score
scores_df = pd.DataFrame([(n, s) for n, s in per_n_scores.items()], columns=['N', 'score'])
scores_df = scores_df.sort_values('score', ascending=False)

print('=== Top 20 N values by score contribution ===')
print(scores_df.head(20))

print('\n=== Score distribution by N range ===')
for start, end in [(1, 10), (11, 30), (31, 50), (51, 100), (101, 150), (151, 200)]:
    mask = (scores_df['N'] >= start) & (scores_df['N'] <= end)
    total = scores_df[mask]['score'].sum()
    print(f'N={start}-{end}: {total:.4f}')

In [ ]:
# Check the jiweiliu approach: SA with translations
# This approach creates structured packings using 2 base trees translated in a grid pattern
# Key insight: For large N (>50), structured packings can be more efficient than random SA

# The approach:
# 1. Start with 2 base trees at angles (e.g., 0 and 180 degrees)
# 2. Translate them in a grid pattern (nx x ny)
# 3. Use SA to optimize the translation distances and angles
# 4. Apply deletion cascade: for each N, try removing each tree and keep the best

print('=== Key insights from jiweiliu kernel ===')
print('1. SA with translations: Create structured packings using 2 base trees')
print('2. Deletion cascade: Propagate good large configs to smaller N')
print('3. Numba + multiprocessing for acceleration')
print('4. ~0.15 improvement in under 2 minutes!')
print('')
print('This is a FUNDAMENTALLY DIFFERENT approach from standard SA!')

In [ ]:
# Check if we have the jiweiliu kernel code available
import json

kernel_path = '/home/code/research/kernels/jiweiliu_super-fast-simulated-annealing-with-translations/super-fast-simulated-annealing-with-translations.ipynb'
if os.path.exists(kernel_path):
    print('jiweiliu kernel is available!')
    with open(kernel_path) as f:
        nb = json.load(f)
    # Count cells
    code_cells = [c for c in nb['cells'] if c['cell_type'] == 'code']
    print(f'Number of code cells: {len(code_cells)}')
else:
    print('jiweiliu kernel not found')

In [ ]:
# The key parameters from jiweiliu kernel:
print('=== Key parameters from jiweiliu kernel ===')
print('SA parameters:')
print('  Tmax: 0.001')
print('  Tmin: 0.000001')
print('  nsteps: 10')
print('  nsteps_per_T: 10000')
print('  position_delta: 0.002')
print('  angle_delta: 1.0')
print('  delta_t: 0.002')
print('')
print('Grid configurations: Automatically explores all viable grid sizes')
print('  - 2 base trees at different angles')
print('  - Translated in nx x ny grid')
print('  - Can add extra trees at edges for non-multiple counts')
print('')
print('Deletion cascade: For each N from 200 down to 2:')
print('  - Try removing each tree from N+1 config')
print('  - Keep the one that minimizes bounding box for N')

In [ ]:
# Check what the evaluator said about parameters
print('=== Evaluator feedback summary ===')
print('1. Parameters used (20,000 iterations, 10 restarts) are 7.5x lower than top kernels')
print('2. Top kernels use 150,000+ iterations, 32+ restarts')
print('3. The v18 optimizer with OpenMP parallelization is more sophisticated')
print('4. The egortrushin SA-with-translations approach is fundamentally different')
print('')
print('=== Recommended next steps ===')
print('1. Implement the jiweiliu SA-with-translations approach')
print('2. This creates structured packings that may escape the local optimum')
print('3. Apply deletion cascade to propagate improvements to smaller N')
print('4. This is a PIVOT to a fundamentally different approach!')